<a href="https://colab.research.google.com/github/s5646s/2024-spark/blob/main/seungwee/movie_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, year, datediff, expr
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import to_date

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [ ]:
movies_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("movieTitle", StringType(), True),
    StructField("releaseDate", StringType(), True),
    StructField("videoReleaseDate", StringType(), True),
    StructField("IMDbURL", StringType(), True)
])

# Genres schema
genres_schema = StructType([
    StructField("mgenreId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("genre", StringType(), True)
])

# Ratings schema
ratings_schema = StructType([
    StructField("ratingId", IntegerType(), True),
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", StringType(), True)
])

# Users schema
users_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("zipCode", StringType(), True)
])

In [ ]:
movie_df = spark.read.csv('movie.csv', header=True, sep=",", schema=movies_schema)
movie_df.show(5, truncate=False)
movie_df.printSchema()

movie_genres_df = spark.read.csv('movie_genres.csv', header=True, sep="," , schema=genres_schema)
movie_genres_df.show(5, truncate=False)
movie_genres_df.printSchema()

ratings_df = spark.read.csv('ratings.csv', header=True, sep=",", schema=ratings_schema)
ratings_df.show(5, truncate=False)
ratings_df.printSchema()

user_df = spark.read.csv('user.csv', header=True, sep=",", schema=users_schema)
user_df.show(5, truncate=False)
user_df.printSchema()

+-------+-----------------+-----------+----------------+------------------------------------------------------+
|movieId|movieTitle       |releaseDate|videoReleaseDate|IMDbURL                                               |
+-------+-----------------+-----------+----------------+------------------------------------------------------+
|1      |Toy Story (1995) |01-Jan-95  |01-Jan-95       |http://us.imdb.com/M/title-exact?Toy%20Story%20(1995) |
|2      |GoldenEye (1995) |01-Jan-95  |01-Jan-95       |http://us.imdb.com/M/title-exact?GoldenEye%20(1995)   |
|3      |Four Rooms (1995)|01-Jan-95  |01-Jan-95       |http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|
|4      |Get Shorty (1995)|01-Jan-95  |01-Jan-95       |http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|
|5      |Copycat (1995)   |01-Jan-95  |01-Jan-95       |http://us.imdb.com/M/title-exact?Copycat%20(1995)     |
+-------+-----------------+-----------+----------------+------------------------------------------------

# **분석1: 특정 사용자 그룹의 영화 선호도 분석**
문제 설명
특정 연령대와 성별의 사용자들이 선호하는 영화 장르를 분석하십시오.

연령대: 20세에서 30세 사이
성별: 여성

1. ratings, movie_genres, user 테이블을 사용합니다.
2. 해당 연령대와 성별의 사용자들이 평가한 영화를 기준으로 분석합니다.
3. 각 장르별 영화의 평균 평점과 영화 수를 계산합니다.
4. 평균 평점이 높은 순서로 결과를 정렬합니다. 동일한 평점일 경우 영화 수가 많은 순서로 정렬합니다.

In [ ]:
user_df_filtered = user_df.filter((col('age') >= 20) & (col('age') <= 30) & (col('gender') == 'F'))

joined_df = ratings_df.join(movie_genres_df, "movieId").join(user_df_filtered, "userId")

result_df = joined_df.groupBy("genre") \
    .agg(count("*").alias("movieCount"), avg("rating").alias("averageRating")) \
    .orderBy(col("averageRating").desc(), col("movieCount").desc())

result_df.show()

+-----------+----------+------------------+
|      genre|movieCount|     averageRating|
+-----------+----------+------------------+
|        War|      1700|3.6941176470588237|
|    Romance|      4878|3.5953259532595325|
|    Musical|      1276|3.5940438871473352|
|  Film-Noir|       268| 3.582089552238806|
|      Drama|      8784| 3.566256830601093|
|  Animation|       994| 3.545271629778672|
|      Crime|      1552| 3.506443298969072|
|Documentary|       138| 3.463768115942029|
|    Mystery|       978|3.4519427402862988|
|  Adventure|      2804| 3.412981455064194|
|   Thriller|      4464|3.4094982078853047|
|     Sci-Fi|      2408|3.3903654485049834|
| Children's|      2038| 3.379784102060844|
|     Action|      4904| 3.372756933115824|
|     Comedy|      7046|3.3624751632131704|
|    Western|       312| 3.217948717948718|
|     Horror|      1142|3.1453590192644483|
|    Fantasy|       330|3.0424242424242425|
|    unknown|         2|               3.0|
+-----------+----------+--------

# **분석2: 고평점 영화와 저평점 영화의 특징 비교**
평균 평점이 높은 영화와 낮은 영화의 장르 분포를 비교하십시오.


1. ratings, movie_genres 테이블을 사용합니다.
2. 먼저 각 영화의 평균 평점을 계산합니다.
3. 평균 평점이 4.0 이상인 영화와 2.0 미만인 영화를 구분하여 장르 분포를 계산합니다.
4. 각 장르별로 고평점 영화와 저평점 영화의 개수를 비교하여 결과를 도출합니다.

In [ ]:
movie_ratings_df = ratings_df.groupBy("movieId") \
    .agg(avg("rating").alias("averageRating"))

joined_df = movie_ratings_df.join(movie_genres_df, "movieId")

result_df = joined_df.groupBy("genre") \
    .agg(
        count(expr("case when averageRating >= 4.0 then 1 end")).alias("highRatedCount"),
        count(expr("case when averageRating < 2.0 then 1 end")).alias("lowRatedCount")
    ) \
    .orderBy(col("highRatedCount").desc(), col("lowRatedCount").desc())

result_df.show()

+-----------+--------------+-------------+
|      genre|highRatedCount|lowRatedCount|
+-----------+--------------+-------------+
|      Drama|           168|           98|
|     Comedy|            72|           74|
|   Thriller|            66|           26|
|    Romance|            54|           18|
|        War|            40|            2|
|      Crime|            34|            4|
|     Action|            32|           36|
|  Adventure|            30|            8|
|    Mystery|            28|            4|
|Documentary|            24|           12|
|  Film-Noir|            24|            2|
|     Sci-Fi|            16|           10|
|  Animation|            10|            4|
|     Horror|             6|           26|
|    Musical|             6|            2|
| Children's|             4|           16|
|    Western|             4|            4|
|    Fantasy|             2|            6|
|    unknown|             0|            2|
+-----------+--------------+-------------+



# **분석3: 사용자 활동 분석**
가장 많이 평점을 매긴 사용자 상위 10명과 그들의 평균 평점을 분석하십시오.

1. ratings 테이블을 사용합니다.
2. 각 사용자가 매긴 평점 수와 평균 평점을 계산합니다.
3. 평점 수가 많은 순서대로 상위 10명의 사용자와 그들의 평균 평점을 도출합니다.

In [ ]:
result_df = ratings_df.groupBy("userId") \
    .agg(count("*").alias("ratingCount"), avg("rating").alias("averageRating")) \
    .orderBy(col("ratingCount").desc()) \
    .limit(10)

result_df.show()

+------+-----------+------------------+
|userId|ratingCount|     averageRating|
+------+-----------+------------------+
|   405|        737|1.8344640434192674|
|   655|        685| 2.908029197080292|
|    13|        636|  3.09748427672956|
|   450|        540|3.8648148148148147|
|   276|        518| 3.465250965250965|
|   416|        493| 3.845841784989858|
|   537|        490|2.8653061224489798|
|   303|        484| 3.365702479338843|
|   234|        480| 3.122916666666667|
|   393|        448|3.3370535714285716|
+------+-----------+------------------+



# **분석4: 특정 장르의 영화들에 대한 평점과 사용자 연령대 분석**
특정 장르의 영화에 대한 연령대별 평균 평점을 분석하십시오.

*   장르: 'Action'

1. ratings, user, movie_genres 테이블을 사용합니다.
2. 'Action' 장르의 영화를 평가한 사용자들의 연령대별 평균 평점을 계산합니다.
3. 연령대별 평균 평점을 연령 순으로 정렬하여 도출합니다.

In [ ]:
joined_df = ratings_df.join(user_df, "userId").join(movie_genres_df, "movieId")

result_df = joined_df.filter(col("genre") == "Action") \
    .groupBy("age") \
    .agg(avg("rating").alias("averageRating")) \
    .orderBy("age")

result_df.show()

+---+------------------+
|age|     averageRating|
+---+------------------+
|  7| 3.611111111111111|
| 10|               4.0|
| 11| 3.076923076923077|
| 13|3.4934210526315788|
| 14| 3.593220338983051|
| 15|               3.5|
| 16|3.5217391304347827|
| 17| 3.651006711409396|
| 18|3.7417677642980935|
| 19| 3.307101727447217|
| 20| 3.696747967479675|
| 21|3.4532760472610096|
| 22|3.2726432532347505|
| 23| 3.193243243243243|
| 24|3.4625199362041466|
| 25|3.5377952755905513|
| 26| 3.243414120126449|
| 27|3.4154676258992804|
| 28|3.4402907580477673|
| 29| 3.422773393461105|
+---+------------------+
only showing top 20 rows



# **분석5: ???**
직접 분석할 것 정의해서 데이터 뽑아보기

위에 주어진 문제들과 유사한 패턴으로 내용을 정의하고 그에맞추어 데이터 추출해보기